In [1]:
def saveData(data, filename):
    #np.savez(filename,data = data)
    output = open(filename, 'wb')
    # Pickle dictionary using protocol 0.
    pickle.dump(data, output)
    output.close()

def loadData(filename):
    pkl_file = open(filename, 'rb')
    data = pickle.load(pkl_file)

    pkl_file.close()
    #loader = np.load(filename)
    return data

In [6]:
import os
import re
import sys
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import scipy.io as sio
import scipy.sparse as sp
from scipy.sparse import csc_matrix
import csv
import matplotlib.pyplot as plt
import datetime
import time
import os
import pickle
from math import radians, cos, sin, asin, sqrt, degrees, atan2, ceil

%matplotlib inline
sys.path.append('../')

In [10]:
trajectory=loadData('../trajectory')
trajectory[0]

array([[1477969147, 104.07513, 30.727240000000002],
       [1477969150, 104.07513, 30.72702],
       [1477969154, 104.07504, 30.72672],
       [1477969156, 104.07497, 30.7263],
       [1477969159, 104.07497, 30.725820000000002],
       [1477969162, 104.07496, 30.725440000000003],
       [1477969168, 104.07489, 30.724870000000003],
       [1477969171, 104.07476, 30.724559999999997],
       [1477969174, 104.07457, 30.72434],
       [1477969177, 104.07434, 30.724059999999998],
       [1477969180, 104.07422, 30.72379],
       [1477969183, 104.07424, 30.723509999999997],
       [1477969186, 104.07449, 30.72315],
       [1477969189, 104.07481, 30.723],
       [1477969193, 104.07506, 30.72298],
       [1477969195, 104.07539, 30.72301],
       [1477969198, 104.07576, 30.72308]], dtype=object)

In [11]:
processed_trajectory = []
for traj in trajectory:
    if len(traj) > 10:
        processed_trajectory.append(traj)
len(processed_trajectory)

196046

In [68]:
def length(traj):  #for LAA space
    return sum(traj[0,:])

def calc_len_angular(ptA, ptB):
    radLatA = radians(ptA[2])
    radLonA = radians(ptA[1])
    radLatB = radians(ptB[2])
    radLonB = radians(ptB[1])

    dLon = radLonB - radLonA
    dLat = radLatB - radLatA
    a = sin(dLat / 2) ** 2 + cos(radLatB) *cos(radLatA) * sin(dLon / 2) ** 2
    c = 2 * asin(sqrt(a))
    d = c * 6371
#     print(i, dLat, dLon, radLatB, radLonB)
    length = d

    y = sin(dLon) * cos(radLatB)
    x = cos(radLatA) * sin(radLatB) - sin(radLatA) * cos(radLatB) * cos(dLon)
    angular = degrees( atan2(y, x))
#     angular = (angular + 360) % 360
#     if angular>180:
#         angular = angular - 360
    
    return length, angular

def transform_len_angular(traj):
    tmp = [[0, 0] for _ in range(len(traj))]
    for i in range(len(traj)):
        if i == 0:
            tmp[i][0] = 0
            tmp[i][1] = 0
        else:
            tmp[i][0], tmp[i][1] = calc_len_angular(traj)
    return np.array(tmp)

In [65]:
def dLAA(vectorA, vectorB):
    return sqrt(vectorA[0]**2 + vectorB[0]**2 - 2 * vectorA[0] * vectorB[0] * cos(radians(vectorA[1] - vectorB[1])))

def distance_dtw(traj_c, traj_q):
    tmp = [[0, 0] for _ in range(len(traj_c))]
    cand = transform_len_angular(traj_c)
    query = transform_len_angular(traj_q)
   
    
    av_cand = sum(cand[1])/len(cand)
    av_query = sum(query[1])/len(query)
    cand[:, 1] = cand[:, 1] - av_cand  #Normalization
    query[:, 1] = query[:, 1] - av_query
    
    n = len(cand)
    m = len(query)
    D0 = np.zeros((n + 1, m + 1))
    D0[0,1:] = np.inf
    D0[1:,0] = np.inf
    D = D0[1:,1:]
    for i in range(n):
        for j in range(m):
            cost =  dLAA(cand[i], query[j])
            D[i,j] = cost + min(D0[i,j], D0[i,j+1], D0[i+1,j])
            #print(self.D)
    return D[-1, -1]

def distance_anchor_aligning(cand, query):
    for i in range(cand):
        
    
    av_cand = sum(cand[1])/len(cand)
    av_query = sum(query[1])/len(query)
    cand[:, 1] = cand[:, 1] - av_cand  #normalization
    query[:, 1] = query[:, 1] - av_query
    
    
    return average_dist

In [ ]:
def ExactS(traj_c, traj_q, minLen):
    subsim = 999999
    subtraj = [0, len(traj_c)-1]
    subset = {}
    N = len(traj_c)
    M = len(traj_q)
    for k in range(M):
        print("stage:", k)
        for m in range(k, M):
            if m-k<minLen:
                continue
            for i in range(N):
                for j in range(i, N):
                    if j-i<minLen:
                        continue
                    len_c = 
                    len_q = 
                    if len_c/len_q>1.5 or len_q/len_c>1.5:
                        continue
                    temp = distance(cand[i:j+1], query[k:m+1])
                    subset[(i, j, k, m)] = temp
                    if temp < subsim:
                        subsim = temp
                        subtraj = (i, j, k, m)
                        print(m-k)
    return subsim, subtraj, subset

In [ ]:
query = transformed_trajectory[2]
cand = transformed_trajectory[1]
minLen = 10

In [ ]:
subsim, subtraj, subset  = ExactS(cand, query, minLen)
print('sub-trajectory', subtraj)
print('sub-similarity', subsim)

In [ ]:
def SBLCS(traj_c, traj_q, minLen, error_dict):
    cand, query = Normalization(traj_c, traj_q)
    m = [[0 for i in range(len(query) + 1)] for j in range(len(cand) + 1)]  # 为方便后续计算，比字符串长度多了一列
    maxx = 0
    subsim = 999999
    subtraj = [0, len(traj_c)-1]
    subset = {}
    for i in range(len(cand)):
        print("stage:", i)
        for j in range(len(query)):
            if abs(cand[i][0] - query[j][0]) < error_dict[0] and abs(cand[i][1] - query[j][1]) < error_dict[1]:
                m[i + 1][j + 1] = m[i][j] + 1
                if m[i+1][j+1]> maxx:
                    maxx = m[i+1][j+1]
                if m[i + 1][j + 1] > minLen:
                    temp = distance(cand[i - m[i + 1][j + 1] + 1: i + 1], query[j - m[i + 1][j + 1] +1 : j + 1])
                    subset[(i - m[i + 1][j + 1] + 1 , i, j - m[i + 1][j + 1] + 1 , j)] = temp
                    if temp < subsim:
                        subsim = temp
                        subtraj = (i - m[i + 1][j + 1] + 1, i, j - m[i + 1][j + 1] + 1, j)
                        print(m[i + 1][j + 1] )
                        print('sub-trajectory', subtraj)
                        print('sub-similarity', subsim)
    return maxx, subsim, subtraj, subset

In [ ]:
error_dict = [0.15, 0.15]
query = transformed_trajectory[3]
cand = transformed_trajectory[13]
print(len(query), len(cand))

In [ ]:
maxx, subsim, subtraj, subset  = SBLCS(cand, query, 10, error_dict) 
# subsim, subtraj, subset = SBLCS(cand, query, 7, error_dict) 
print('sub-trajectory', subtraj)
print('sub-similarity', subsim)
print("maxlen:",maxx)

In [ ]:
def value2symbol(value, table):
    tmp = 0
    vocab = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
    for idx in range(len(table[0])):
        if value[0] > table[0][idx]:
            tmp += 1
    symbol = vocab[tmp]
    tmp = 0
    for idx in range(len(table[1])):
        if value[1] > table[1][idx]:
            tmp += 1
    symbol = symbol +vocab[tmp]
    return symbol
    
def generate_symbols(cand, query):
    alpha_table = [[0.2, 0.4, 0.6, 0.8], [-0.5, -0.3, -0.15, 0, 0.15, 0.3, 0.5]]
    symbol_cand = []
    symbol_query = []
    for i in range(len(cand)):
        symbol_cand.append(value2symbol(cand[i], alpha_table))
    for j in range(len(query)):
        symbol_query.append(value2symbol(query[j], alpha_table))
    return symbol_cand, symbol_query

def SBSAX(traj_c, traj_q, minLen):
    cand, query = Normalization(traj_c, traj_q)
    symbol_c, symbol_q = generate_symbols(traj_c, traj_q)
    subsim = 999999
    subtraj = [0, len(traj_c)-1]
    subset = {}
    m = [[0 for i in range(len(query) + 1)] for j in range(len(cand) + 1)]
    maxx = 0

    for i in range(len(symbol_c)):
        print("stage:", i)
        for j in range(len(symbol_q)):            
            if symbol_c[i] == symbol_q[j]:
                m[i + 1][j + 1] = m[i][j] + 1
                if m[i+1][j+1]  > maxx:             
                    maxx = m[i+1][j+1]
                if m[i + 1][j + 1] > minLen :
                    temp = distance(cand[i - m[i + 1][j + 1] + 1: i + 1], query[j - m[i + 1][j + 1] +1 : j + 1])
                    subset[(i - m[i + 1][j + 1] + 1 , i, j - m[i + 1][j + 1] + 1 , j)] = temp
                    if temp < subsim:
                        subsim = temp
                        subtraj = (i - m[i + 1][j + 1] + 1, i, j - m[i + 1][j + 1] + 1, j)
                        print(m[i + 1][j + 1] )
                        print('sub-trajectory', subtraj)
                        print('sub-similarity', subsim)
    return maxx, subsim, subtraj, subset

In [ ]:
minLen = 10
maxx, subsim, subtraj, subset  = SBSAX(cand, query, minLen) 
# subsim, subtraj, subset = SBLCS(cand, query, 7, error_dict) 
print('sub-trajectory', subtraj)
print('sub-similarity', subsim)
print("maxlen:",maxx)

In [ ]:
def SBHierachical(traj_c, traj_q, minLen, error_dict):
    cand, query = Normalization(traj_c, traj_q)
    m = [[0 for i in range(len(query) + 1)] for j in range(len(cand) + 1)]  # 为方便后续计算，比字符串长度多了一列
    maxx = 0
    subsim = 999999
    subtraj = [0, len(traj_c)-1]
    subset = {}
    for i in range(len(cand)):
        print("stage:", i)
        for j in range(len(query)):
            if abs(cand[i][0] - query[j][0]) < error_dict[0] and abs(cand[i][1] - query[j][1]) < error_dict[1]:
                m[i + 1][j + 1] = m[i][j] + 1
                if m[i+1][j+1] > maxx:
                    maxx = m[i+1][j+1]
                if m[i + 1][j + 1] > minLen:
                    temp = distance(cand[i - m[i + 1][j + 1] + 1: i + 1], query[j - m[i + 1][j + 1] +1 : j + 1])
                    subset[(i - m[i + 1][j + 1] + 1 , i, j - m[i + 1][j + 1] + 1 , j)] = temp
                    if temp < subsim:
                        subsim = temp
                        subtraj = (i - m[i + 1][j + 1] + 1, i, j - m[i + 1][j + 1] + 1, j)
                        print(m[i + 1][j + 1] )
                        print('sub-trajectory', subtraj)
                        print('sub-similarity', subsim)
    return maxx, subsim, subtraj, subset